<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0616/blob/main/tibame_night_20250708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
HTTP代碼
1. 2: 成功
2. 3: 轉址
3. 4: 錯誤
403 Forbidden
1. IP被ban: 換ip/等
2. 模仿不夠像: 缺少必要headers
```





```
如果你的html區塊是沒有class/id可以讓你篩選的
1. 找他老爸(必須有class/id)
2. 自己想辦法寫一個篩選方式(文字/超連結網址)

```



In [ ]:
import urllib.request as req

# 相對路徑:
# a.txt: 現在這個目錄下的a.txt
# ./a.txt: 現在(.)這個目錄下的a.txt
# ../a.txt: 前一層目錄(..)的a.txt
def get_imgur_image(url, dirname="."):
    # dirname = "file"
    # url = "https://i.imgur.com/OSwXeuz.jpeg"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
    resp = req.urlopen(r)
    content = resp.read()

    # 檔案讀寫
    # 1. 純文字: r/w  encoding="utf-8"
    # 2. 非純文字(jpeg, png, docx, xlsx, pdf...): rb/wb
    fp = dirname + "/" + url.split("/")[-1]
    print("存放位置:", fp)
    with open(fp, "wb") as f:
        f.write(content)

# url = "https://i.imgur.com/H8i2ZVT.jpeg"
# get_imgur_image(url, "test")

In [ ]:
import json
import os
import urllib.request as req
import bs4 as bs

def download_ptt_article(url, root="."):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
    # 儲存的路徑
    dirname = url.split("/")[-1]
    dirname = os.path.join(root, dirname)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
    resp = req.urlopen(r)
    content = resp.read()
    html = bs.BeautifulSoup(content)
    # find/find_all
    # .text .get_text() ["href"]

    # 作者/標題/時間/看板
    metas = html.find_all("span", {"class":"article-meta-value"})
    author = metas[0]
    board = metas[1]
    title = metas[2]
    post_time = metas[3]
    pushes = html.find_all("div", {"class":"push"})


    author_text = author.text
    board_text = board.text
    title_text = title.text
    post_time_text = post_time.text

    push_list = []
    for p in pushes:
        push_meta = p.find_all("span")
        push_trans = {"推":1, "噓":-1, "→":0}
        push_data = {
            "type":push_trans[push_meta[0].text.strip()],
            "uid":push_meta[1].text.strip(),
            "text":push_meta[2].text.strip().replace(": ", ""),
            "ipdatetime":push_meta[3].text.strip()
        }
        push_list.append(push_data)

    # 下載所有圖片
    allow_sub = ["jpg", "jpeg", "png", "gif"]
    links = html.find_all("a")
    for l in links:
        href = l["href"]
        sub = href.split(".")[-1]
        if sub.lower() in allow_sub:
            print("符合格式, 下載:", href)
            get_imgur_image(href, dirname)

    # 內文: 沒有任何區塊把內文包起來, 先找到大區塊, 把區塊裡面你不要的東西去掉(extract)
    main_content = html.find("div", {"id":"main-content"})
    for e in html.find_all("div", {"class":"article-metaline"}):
        e.extract()
    for e in html.find_all("div", {"class":"article-metaline-right"}):
        e.extract()
    for e in html.find_all("div", {"class":"push"}):
        e.extract()
    # print(main_content)
    content_text = main_content.text.strip()


    row = {
        "author":author_text,
        "title":title_text,
        "post_time":post_time_text,
        "board":board_text,
        "pushes":push_list,
        "content":content_text,
    }
    # print(row)
    fn = os.path.join(dirname, "data.json")
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(row, f, indent=4, ensure_ascii=False)

# test code
url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
download_ptt_article(url, "Beauty")

符合格式, 下載: https://i.imgur.com/OSwXeuz.jpeg
存放位置: Beauty/M.1751797464.A.265.html/OSwXeuz.jpeg
符合格式, 下載: https://i.imgur.com/c2CmTdy.jpeg
存放位置: Beauty/M.1751797464.A.265.html/c2CmTdy.jpeg
符合格式, 下載: https://i.imgur.com/WDzTRGh.jpeg
存放位置: Beauty/M.1751797464.A.265.html/WDzTRGh.jpeg
符合格式, 下載: https://i.imgur.com/Ue2jqqf.jpeg
存放位置: Beauty/M.1751797464.A.265.html/Ue2jqqf.jpeg
符合格式, 下載: https://i.imgur.com/ejEwGXA.jpeg
存放位置: Beauty/M.1751797464.A.265.html/ejEwGXA.jpeg
符合格式, 下載: https://i.imgur.com/H8i2ZVT.jpeg
存放位置: Beauty/M.1751797464.A.265.html/H8i2ZVT.jpeg
符合格式, 下載: https://i.imgur.com/Hhc8hwu.gif
存放位置: Beauty/M.1751797464.A.265.html/Hhc8hwu.gif


In [ ]:
import urllib.request as req
import bs4 as bs


url = "https://www.ptt.cc/bbs/Beauty/index4004.html"
dirname = "Beauty"

# 4004
# Beauty - M..xxxx.html - jpeg/png
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)

posts = html.find_all("div", {"class":"title"})
for post in posts:
    post_a = post.find("a")
    if not post_a == None:
        if not "公告" in post_a.text:
            post_a_href = "https://www.ptt.cc" + post_a["href"]
            print(post_a_href)
            download_ptt_article(post_a_href, dirname)

https://www.ptt.cc/bbs/Beauty/M.1752121564.A.A85.html
符合格式, 下載: https://i.imgur.com/8JJrkOG.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/8JJrkOG.jpeg
符合格式, 下載: https://i.imgur.com/2Q1VRwa.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/2Q1VRwa.jpeg
符合格式, 下載: https://i.imgur.com/lNsVi0W.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/lNsVi0W.jpeg
符合格式, 下載: https://i.imgur.com/cB3HMEh.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/cB3HMEh.jpeg
符合格式, 下載: https://i.imgur.com/x3ysLsW.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/x3ysLsW.jpeg
符合格式, 下載: https://i.imgur.com/35O5eF3.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/35O5eF3.jpeg
符合格式, 下載: https://i.imgur.com/gVfWz3m.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/gVfWz3m.jpeg
符合格式, 下載: https://i.imgur.com/F5Q4C9p.jpeg
存放位置: drive/My Drive/Beauty/M.1752121564.A.A85.html/F5Q4C9p.jpeg
符合格式, 下載: https://i.imgur.com/IXQsKvR.jpeg
存放位置: drive/My Drive/Beauty/M.175212156

In [ ]:
f = open("a.txt", "w", encoding="utf-8")
f.write("abc")
f.close()

with open("a.txt", "w", encoding="utf-8") as f:
    f.write("abc")

In [ ]:
import os

dirname = "a/b/c/d"
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [ ]:
# 資料夾路徑如果是用字串的相加其實很危險
# 萬一沒打斜線/或者打了兩個斜線
os.path.join("a/", "b")
os.path.join("a", "b/", "c", "d")

'a/b/c/d'

In [ ]:
# 如果你想把你的成果搞到local
!zip -r /content/file.zip /content/Beauty

In [23]:
# 會變: *
import glob
import json
import pandas as pd

table = []
fp_list = glob.glob("Beauty/M.*/data.json")
for fp in fp_list:
    f = open(fp, "r", encoding="utf-8")
    content = f.read()
    f.close()
    row = json.loads(content)
    table.append(row)
df = pd.DataFrame(table)
df

,author,title,post_time,board,pushes,content
0,yokann (我怕練太壯),[正妹] 桃桃,Thu Jul 10 02:20:49 2025,Beauty,"[{'type': 1, 'uid': 'lsj049', 'text': '長髮短髮都很香...",https://i.imgur.com/o7A1smt.jpeg\nhttps://i.im...
1,AmedRosario (),[正妹] 瑜珈老師,Wed Jul 9 12:00:10 2025,Beauty,"[{'type': 1, 'uid': 'maxxxxxx', 'text': '正', '...",https://i.meee.com.tw/ThWlQKE.jpg\nhttps://i.m...
2,Kjartan (Olafsson),[正妹] Diana 以色列女孩,Thu Jul 10 20:19:05 2025,Beauty,"[{'type': 1, 'uid': 'johnwu', 'text': '軍裝會把奶子藏...",Diana 以色列女孩\n\nhttps://images2.imgbox.com/79/...
3,ckpot (ckpot_plussizetw),[正妹] 大尺碼｜肉特(1135),Wed Jul 9 21:01:47 2025,Beauty,[],Ludmilla Holanda｜BR\n\n\nhttps://i.meee.com.tw...
4,yokann (我怕練太壯),[正妹] 誠意滿滿,Thu Jul 10 02:09:41 2025,Beauty,"[{'type': 1, 'uid': 'amare1015', 'text': '只管看這...",https://i.imgur.com/bPRvgNG.jpeg\nhttps://i.im...
5,jerryyuan (財運滾滾),[正妹] 蔡秀彬,Thu Jul 10 12:26:00 2025,Beauty,"[{'type': 1, 'uid': 'sp123008', 'text': 'Push'...",今日壽星~生日快樂！\n\nhttps://i.imgur.com/8JJrkOG.jpeg...
6,jerryyuan (財運滾滾),[正妹] 李多慧,Wed Jul 9 22:05:48 2025,Beauty,"[{'type': 1, 'uid': 'chemicaltom', 'text': '越來...",https://i.imgur.com/e2qL494.jpeg\nhttps://i.im...
7,jerryyuan (財運滾滾),[帥哥] 黃少祺,Sun Jul 6 18:23:37 2025,Beauty,"[{'type': 1, 'uid': 'H132196', 'text': '神機妙算劉伯...",54歲~生日快樂!\n\nhttps://i.imgur.com/OSwXeuz.jpeg\...
8,yavakaka (爛兵683T),[正妹] 普但大,Thu Jul 10 17:29:05 2025,Beauty,"[{'type': 1, 'uid': 'sulacow', 'text': '不錯呀!',...",https://i.postimg.cc/VNbGKd9Q/S-63643692-0.jpg...
9,Kjartan (Olafsson),[正妹] Giusy Piazza 義大利女孩,Thu Jul 10 13:02:42 2025,Beauty,"[{'type': 0, 'uid': 'hornybaron', 'text': '貴死披...",Giusy Piazza 義大利女孩\n\nhttps://images2.imgbox....


In [ ]:
# import!!!

# import 一隻.py
import urllib.request
urllib.request.urlopen
# from .. import
from urllib import request
request.urlopen
# 特異功能: from .py import 功能
# 優點: 可讀性 缺點: 真的就只能用open
from urllib.request import urlopen
urlopen


In [ ]:
from datetime import datetime

def flow(s):
    # s = "Thu Jul 10 07:13:33 2025"
    pat = "%a %b %d %H:%M:%S %Y"
    # 轉換成datetime型態
    dt = datetime.strptime(s, pat)
    pat_convert = "%Y/%m/%d %H:%M:%S"
    # 轉換成str型態
    dt_s = dt.strftime(pat_convert)
    return dt_s
df["post_time"] = df["post_time"].apply(flow)
df

In [35]:
# pat = "%Y/%m/%d %H:%M:%S"
# dt1 = datetime.strptime("2025/07/10 07:13:33", pat)
# dt2 = datetime.strptime("2025/07/10 06:13:33", pat)
# dt2 > dt1
# apply做出跟我資料筆數依樣多的True/False來過濾
def flow(s):
    pat = "%Y/%m/%d %H:%M:%S"
    dt = datetime.strptime(s, pat)
    target = datetime.strptime("2025/07/10 00:00:00", pat)
    return dt > target
fil = df["post_time"].apply(flow)
df[fil]

,author,title,post_time,board,pushes,content
0,yokann (我怕練太壯),[正妹] 桃桃,2025/07/10 02:20:49,Beauty,"[{'type': 1, 'uid': 'lsj049', 'text': '長髮短髮都很香...",https://i.imgur.com/o7A1smt.jpeg\nhttps://i.im...
2,Kjartan (Olafsson),[正妹] Diana 以色列女孩,2025/07/10 20:19:05,Beauty,"[{'type': 1, 'uid': 'johnwu', 'text': '軍裝會把奶子藏...",Diana 以色列女孩\n\nhttps://images2.imgbox.com/79/...
4,yokann (我怕練太壯),[正妹] 誠意滿滿,2025/07/10 02:09:41,Beauty,"[{'type': 1, 'uid': 'amare1015', 'text': '只管看這...",https://i.imgur.com/bPRvgNG.jpeg\nhttps://i.im...
5,jerryyuan (財運滾滾),[正妹] 蔡秀彬,2025/07/10 12:26:00,Beauty,"[{'type': 1, 'uid': 'sp123008', 'text': 'Push'...",今日壽星~生日快樂！\n\nhttps://i.imgur.com/8JJrkOG.jpeg...
8,yavakaka (爛兵683T),[正妹] 普但大,2025/07/10 17:29:05,Beauty,"[{'type': 1, 'uid': 'sulacow', 'text': '不錯呀!',...",https://i.postimg.cc/VNbGKd9Q/S-63643692-0.jpg...
9,Kjartan (Olafsson),[正妹] Giusy Piazza 義大利女孩,2025/07/10 13:02:42,Beauty,"[{'type': 0, 'uid': 'hornybaron', 'text': '貴死披...",Giusy Piazza 義大利女孩\n\nhttps://images2.imgbox....
11,CavendishJr (梅西務實梅西可愛),[正妹] 已婚,2025/07/10 07:13:33,Beauty,"[{'type': 1, 'uid': 'js52666', 'text': '一定搞出一支...",https://i.imgur.com/SoiAdXc.jpeg\nhttps://i.im...
17,YuiLover (羽冰漾),[正妹] 榎原依那,2025/07/10 13:04:14,Beauty,"[{'type': 0, 'uid': 'YDSK', 'text': 'CHODA!', ...",https://i.meee.com.tw/K2Qz8Lq.jpg\nhttps://i.m...
19,Gentlemon (肥宅紳士),[正妹] Cosplay 2101 韓國 星鐵 檔案,2025/07/10 06:54:07,Beauty,[],1.https://i.imgur.com/ZfGtMmg.jpeg\n2.https://...
20,yavakaka (爛兵683T),[正妹] 愛狗人士,2025/07/10 17:19:55,Beauty,"[{'type': 1, 'uid': 'matt4609', 'text': '汪汪！',...",https://i.postimg.cc/fL9KfpNw/S-63643658-0.jpg...


In [ ]:
# 解釋os.path/requests